# GIQL operators demo using DuckDB query engine

This notebook demonstrates the 6 core GIQL operators:
- **Binary predicates**: INTERSECTS, WITHIN, CONTAINS
- **Aggregation operators**: MERGE, CLUSTER
- **UDF operators**: DISTANCE

For each operator, we:
1. Create a GIQL query
2. Transpile it to standard SQL using GIQL
3. Execute the SQL using polars-bio

In [1]:
import polars as pl
# from sql_formatter.core import format_sql
def format_sql(sql):
    return sql
from giql import GIQLEngine

## Load BED files

Load the two ENCODE ChIP-seq peak files as Polars DataFrames.

In [2]:
# Load BED files with standard BED column names
bed_schema = {
    "chromosome": pl.Utf8,
    "start_pos": pl.Int64,
    "end_pos": pl.Int64,
    "name": pl.Utf8,
    "score": pl.Int64,
    "strand": pl.Utf8,
    "signal_value": pl.Float64,
    "p_value": pl.Float64,
    "q_value": pl.Float64,
    "peak": pl.Int64
}

# Load first BED file
features_a = pl.read_csv(
    ".ENCFF199YFA.bed",
    separator="\t",
    has_header=False,
    schema=bed_schema
)

# Load second BED file
features_b = pl.read_csv(
    ".ENCFF205OKL.bed",
    separator="\t",
    has_header=False,
    schema=bed_schema
)

print(f"Features A: {len(features_a)} intervals")
print(f"Features B: {len(features_b)} intervals")
print("\nFeatures A preview:")
features_a.head()

Features A: 60893 intervals
Features B: 46196 intervals

Features A preview:


chromosome,start_pos,end_pos,name,score,strand,signal_value,p_value,q_value,peak
str,i64,i64,str,i64,str,f64,f64,f64,i64
"""chr17""",27381032,27381306,""".""",1000,""".""",461.10586,-1.0,5.08726,136
"""chr7""",25565932,25566199,""".""",1000,""".""",457.60096,-1.0,5.08726,133
"""chr7""",143215117,143215391,""".""",1000,""".""",443.34712,-1.0,5.08726,139
"""chr2""",164600030,164600306,""".""",1000,""".""",442.64446,-1.0,5.08726,137
"""chr15""",56246046,56246359,""".""",1000,""".""",438.56423,-1.0,5.08726,154


## Setup GIQL engine

Create a GIQL engine and register the table schemas.

In [3]:
# Create GIQL engine with DuckDB backend
engine = GIQLEngine(target_dialect="duckdb", verbose=False)

# Register DataFrames
engine.conn.register("features_a", features_a.to_pandas())
engine.conn.register("features_b", features_b.to_pandas())

# Register schemas with genomic columns
schema_dict = {
    "chromosome": "VARCHAR",
    "start_pos": "BIGINT",
    "end_pos": "BIGINT",
    "name": "VARCHAR",
    "score": "BIGINT",
    "strand": "VARCHAR",
    "signal_value": "DOUBLE",
    "p_value": "DOUBLE",
    "q_value": "DOUBLE",
    "peak": "BIGINT"
}

for table in ["features_a", "features_b"]:
    engine.register_table_schema(
        table,
        schema_dict,
        genomic_column="position"
    )

print("GIQL engine configured successfully")

GIQL engine configured successfully


---

## 1. INTERSECTS operator

Find features in A that **overlap** with features in B.

**GIQL Query**: `SELECT a.* FROM features_a a, features_b b WHERE a.position INTERSECTS b.position`

In [4]:
# Define GIQL query
giql_query = """
    SELECT DISTINCT a.*
    FROM features_a a, features_b b
    WHERE a.position INTERSECTS b.position
"""

# Transpile to SQL
sql = engine.transpile(giql_query)
print("Transpiled SQL:")
print(format_sql(sql))
print("\n" + "="*80 + "\n")

# Execute with DuckDB via GIQL engine
cursor = engine.conn.execute(sql)
result = cursor.df()  # Get result as pandas DataFrame
print(f"Result: {len(result)} overlapping intervals from A")
result.head(10)

Transpiled SQL:
SELECT DISTINCT a.* FROM features_a AS a, features_b AS b WHERE (a."chromosome" = b."chromosome" AND a."start_pos" < b."end_pos" AND a."end_pos" > b."start_pos")


Result: 42616 overlapping intervals from A


,chromosome,start_pos,end_pos,name,score,strand,signal_value,p_value,q_value,peak
0,chr15,65385673,65385878,.,1000,.,241.11271,-1.0,5.08726,101
1,chrX,10119421,10119886,.,1000,.,293.00157,-1.0,5.08726,239
2,chr14,105045741,105046036,.,1000,.,345.30019,-1.0,5.08726,140
3,chr9,135440665,135440912,.,1000,.,302.05379,-1.0,5.08726,129
4,chr20,2093281,2093476,.,1000,.,264.56371,-1.0,5.08726,94
5,chr8,20231850,20232042,.,1000,.,245.11188,-1.0,5.08726,97
6,chr14,24216011,24216203,.,1000,.,251.87452,-1.0,5.08726,95
7,chr14,23380009,23380228,.,1000,.,303.14425,-1.0,5.08726,114
8,chr5,181184125,181184440,.,1000,.,277.71855,-1.0,5.08726,138
9,chr16,9008834,9009050,.,1000,.,262.70537,-1.0,5.08726,116


### High overlap depth regions

Find regions with high overlap depth (where many intervals overlap) using a self-join.

In [5]:
# Define GIQL query to find high-depth overlaps
giql_query = """
    WITH overlap_counts AS (
        SELECT 
            a.chromosome,
            a.start_pos,
            a.end_pos,
            a.signal_value,
            COUNT(*) as depth
        FROM features_b a
        INNER JOIN features_a b ON a.position INTERSECTS b.position
        GROUP BY a.chromosome, a.start_pos, a.end_pos, a.signal_value
    )
    SELECT *
    FROM overlap_counts
    WHERE depth >= 2
    ORDER BY depth DESC
"""

# Transpile to SQL
sql = engine.transpile(giql_query)
print("Transpiled SQL:")
print(format_sql(sql))
print("\n" + "="*80 + "\n")

# Execute with DuckDB via GIQL engine
cursor = engine.conn.execute(sql)
result = cursor.df()  # Get result as pandas DataFrame
print(f"Result: {len(result)} intervals with overlap depth >= 5")
print(f"Max overlap depth: {result['depth'].max() if len(result) > 0 else 0}")
result.head(10)

Transpiled SQL:
WITH overlap_counts AS (SELECT a.chromosome, a.start_pos, a.end_pos, a.signal_value, COUNT(*) AS depth FROM features_b AS a INNER JOIN features_a AS b ON (a."chromosome" = b."chromosome" AND a."start_pos" < b."end_pos" AND a."end_pos" > b."start_pos") GROUP BY a.chromosome, a.start_pos, a.end_pos, a.signal_value) SELECT * FROM overlap_counts WHERE depth >= 2 ORDER BY depth DESC


Result: 109 intervals with overlap depth >= 5
Max overlap depth: 2


,chromosome,start_pos,end_pos,signal_value,depth
0,chr12,453148,453458,70.84404,2
1,chr22,35043452,35043762,13.39651,2
2,chr2,113979698,113980008,35.97698,2
3,chr20,8443523,8443833,18.80028,2
4,chr2,170928770,170929192,13.04917,2
5,chr17,45971661,45971971,18.47366,2
6,chr6,132279569,132280035,151.44008,2
7,chr11,65932320,65932630,11.78241,2
8,chr1,84620782,84621092,33.19524,2
9,chr20,23799344,23799654,65.95268,2


---

## 2. WITHIN operator

Find features in A that are **completely contained within** features in B.

**GIQL Query**: `SELECT a.* FROM features_a a, features_b b WHERE a.position WITHIN b.position`

In [6]:
# Define GIQL query
giql_query = """
    SELECT DISTINCT a.*
    FROM features_a a, features_b b
    WHERE a.position WITHIN b.position
"""

# Transpile to SQL
sql = engine.transpile(giql_query)
print("Transpiled SQL:")
print(format_sql(sql))
print("\n" + "="*80 + "\n")

# Execute with DuckDB via GIQL engine
cursor = engine.conn.execute(sql)
result = cursor.df()  # Get result as pandas DataFrame
print(f"Result: {len(result)} intervals from A contained within B")
result.head(10)

Transpiled SQL:
SELECT DISTINCT a.* FROM features_a AS a, features_b AS b WHERE (a."chromosome" = b."chromosome" AND a."start_pos" >= b."start_pos" AND a."end_pos" <= b."end_pos")


Result: 30806 intervals from A contained within B


,chromosome,start_pos,end_pos,name,score,strand,signal_value,p_value,q_value,peak
0,chr20,52937930,52938139,.,1000,.,242.96853,-1.0,5.08726,110
1,chr16,81403612,81403816,.,1000,.,244.89039,-1.0,5.08726,103
2,chr20,3250135,3250335,.,1000,.,274.74729,-1.0,5.08726,95
3,chr9,92634770,92634990,.,1000,.,313.09355,-1.0,5.08726,113
4,chr9,33722162,33722428,.,1000,.,391.35401,-1.0,5.08726,132
5,chr8,22909069,22909276,.,1000,.,245.13028,-1.0,5.08726,107
6,chr4,182925334,182925586,.,1000,.,284.15090,-1.0,5.08726,135
7,chr17,51322469,51322723,.,1000,.,383.45857,-1.0,5.08726,127
8,chr5,43627096,43627315,.,1000,.,304.51285,-1.0,5.08726,115
9,chr11,2419724,2419926,.,1000,.,247.25473,-1.0,5.08726,104


---

## 3. CONTAINS operator

Find features in A that **completely contain** features in B.

**GIQL Query**: `SELECT a.* FROM features_a a, features_b b WHERE a.position CONTAINS b.position`

In [7]:
# Define GIQL query
giql_query = """
    SELECT DISTINCT a.*
    FROM features_a a, features_b b
    WHERE a.position CONTAINS b.position
"""

# Transpile to SQL
sql = engine.transpile(giql_query)
print("Transpiled SQL:")
print(format_sql(sql))
print("\n" + "="*80 + "\n")

# Execute with DuckDB via GIQL engine
cursor = engine.conn.execute(sql)
result = cursor.df()  # Get result as pandas DataFrame
print(f"Result: {len(result)} intervals from A that contain B")
result.head(10)

Transpiled SQL:
SELECT DISTINCT a.* FROM features_a AS a, features_b AS b WHERE (a."chromosome" = b."chromosome" AND a."start_pos" <= b."start_pos" AND a."end_pos" >= b."end_pos")


Result: 4917 intervals from A that contain B


,chromosome,start_pos,end_pos,name,score,strand,signal_value,p_value,q_value,peak
0,chr20,3218576,3218780,.,1000,.,256.77607,-1.0,5.08726,106
1,chr14,106883188,106883490,.,1000,.,311.14753,-1.0,5.08726,146
2,chrX,10119421,10119886,.,1000,.,293.00157,-1.0,5.08726,239
3,chr12,63843868,63844141,.,1000,.,270.83756,-1.0,5.08726,130
4,chr16,81403612,81403816,.,1000,.,244.89039,-1.0,5.08726,103
5,chr9,137236518,137236806,.,1000,.,323.80804,-1.0,5.08726,138
6,chr15,77633500,77633787,.,1000,.,380.81693,-1.0,5.08726,140
7,chr9,135440665,135440912,.,1000,.,302.05379,-1.0,5.08726,129
8,chr16,68510220,68510505,.,1000,.,409.28654,-1.0,5.08726,133
9,chr8,20231850,20232042,.,1000,.,245.11188,-1.0,5.08726,97


---

## 4. MERGE operator

**Combine overlapping intervals** from features_a into merged regions.

Similar to `bedtools merge`, this collapses overlapping genomic intervals.

**GIQL Query**: `SELECT MERGE(position) FROM features_a`

In [8]:
# Define GIQL query - basic merge
giql_query = """
    SELECT MERGE(position)
    FROM features_b
"""

# Transpile to SQL
sql = engine.transpile(giql_query)
print("Transpiled SQL:")
print(format_sql(sql))
print("\n" + "="*80 + "\n")

# Execute with DuckDB via GIQL engine
cursor = engine.conn.execute(sql)
result = cursor.df()  # Get result as pandas DataFrame
print(f"Result: {len(result)} merged intervals (from {len(features_a)} original)")
result.head(10)

Transpiled SQL:
SELECT "chromosome", MIN("start_pos") AS start_pos, MAX("end_pos") AS end_pos FROM (SELECT *, SUM(is_new_cluster) OVER (PARTITION BY "chromosome" ORDER BY "start_pos" NULLS LAST) AS __giql_cluster_id FROM (SELECT *, CASE WHEN LAG("end_pos") OVER (PARTITION BY "chromosome" ORDER BY "start_pos" NULLS LAST) >= "start_pos" THEN 0 ELSE 1 END AS is_new_cluster FROM features_b) AS lag_calc) AS clustered GROUP BY chromosome, __giql_cluster_id ORDER BY "chromosome" NULLS LAST, "start_pos" NULLS LAST


Result: 45630 merged intervals (from 60893 original)


,chromosome,start_pos,end_pos
0,chr1,186654,186964
1,chr1,267979,268128
2,chr1,850424,850734
3,chr1,869711,870021
4,chr1,912864,913174
5,chr1,919635,919945
6,chr1,931626,931936
7,chr1,938195,938352
8,chr1,951414,951724
9,chr1,976080,976390


### MERGE with aggregation CTE

Compute statistics on merged intervals and filter for intervals that merged multiple features.

In [9]:
# Define GIQL query with aggregations in a CTE
giql_query = """
    WITH merged_intervals AS (
        SELECT
            MERGE(position),
            COUNT(*) as interval_count,
            AVG(signal_value) as avg_signal
        FROM features_b
    )
    SELECT *
    FROM merged_intervals
    WHERE interval_count > 1
"""

# Transpile to SQL
sql = engine.transpile(giql_query)
print("Transpiled SQL:")
print(format_sql(sql))
print("\n" + "="*80 + "\n")

# Execute with DuckDB via GIQL engine
cursor = engine.conn.execute(sql)
result = cursor.df()  # Get result as pandas DataFrame
print(f"Result: {len(result)} merged intervals with more than 1 original interval")
result.head(10)

Transpiled SQL:
WITH merged_intervals AS (SELECT "chromosome", MIN("start_pos") AS start_pos, MAX("end_pos") AS end_pos, COUNT(*) AS interval_count, AVG(signal_value) AS avg_signal FROM (SELECT *, SUM(is_new_cluster) OVER (PARTITION BY "chromosome" ORDER BY "start_pos" NULLS LAST) AS __giql_cluster_id FROM (SELECT *, CASE WHEN LAG("end_pos") OVER (PARTITION BY "chromosome" ORDER BY "start_pos" NULLS LAST) >= "start_pos" THEN 0 ELSE 1 END AS is_new_cluster FROM features_b) AS lag_calc) AS clustered GROUP BY chromosome, __giql_cluster_id ORDER BY "chromosome" NULLS LAST, "start_pos" NULLS LAST) SELECT * FROM merged_intervals WHERE interval_count > 1


Result: 559 merged intervals with more than 1 original interval


,chromosome,start_pos,end_pos,interval_count,avg_signal
0,chr1,1300073,1300592,2,14.844510
1,chr1,8374912,8375330,2,28.608395
2,chr1,9717307,9717819,2,13.833490
3,chr1,14782921,14783438,2,7.107920
4,chr1,18362141,18362677,2,9.312525
5,chr1,19445773,19446283,2,16.590025
6,chr1,23958874,23959385,2,10.616965
7,chr1,24157968,24158482,2,11.985625
8,chr1,27392817,27393359,2,11.789830
9,chr1,30106143,30106664,2,31.694800


### MERGE with distance parameter

Merge intervals within 1000bp of each other.

In [10]:
# Define GIQL query - merge with distance parameter
giql_query = """
    SELECT MERGE(position, 10000)
    FROM features_b
"""

# Transpile to SQL
sql = engine.transpile(giql_query)
print("Transpiled SQL:")
print(format_sql(sql))
print("\n" + "="*80 + "\n")

# Execute with DuckDB via GIQL engine
cursor = engine.conn.execute(sql)
result = cursor.df()  # Get result as pandas DataFrame
print(f"Result: {len(result)} merged intervals (1000bp distance)")
print(f"Compared to: {len(features_a)} original intervals")
result.head(10)

Transpiled SQL:
SELECT "chromosome", MIN("start_pos") AS start_pos, MAX("end_pos") AS end_pos FROM (SELECT *, SUM(is_new_cluster) OVER (PARTITION BY "chromosome" ORDER BY "start_pos" NULLS LAST) AS __giql_cluster_id FROM (SELECT *, CASE WHEN LAG("end_pos") OVER (PARTITION BY "chromosome" ORDER BY "start_pos" NULLS LAST) + 10000 >= "start_pos" THEN 0 ELSE 1 END AS is_new_cluster FROM features_b) AS lag_calc) AS clustered GROUP BY chromosome, __giql_cluster_id ORDER BY "chromosome" NULLS LAST, "start_pos" NULLS LAST


Result: 34097 merged intervals (1000bp distance)
Compared to: 60893 original intervals


,chromosome,start_pos,end_pos
0,chr1,186654,186964
1,chr1,267979,268128
2,chr1,850424,850734
3,chr1,869711,870021
4,chr1,912864,919945
5,chr1,931626,938352
6,chr1,951414,951724
7,chr1,976080,984477
8,chr1,1013987,1015656
9,chr1,1063787,1064097


---

## 5. CLUSTER operator

**Assign cluster IDs** to overlapping intervals from features_a.

Similar to `bedtools cluster`, this groups overlapping genomic intervals.

**GIQL Query**: `SELECT *, CLUSTER(position) AS cluster_id FROM features_a`

In [11]:
# Define GIQL query - basic clustering
giql_query = """
    SELECT
        chromosome,
        start_pos,
        end_pos,
        signal_value,
        CLUSTER(position) AS cluster_id
    FROM features_a
    ORDER BY chromosome, start_pos
"""

# Transpile to SQL
sql = engine.transpile(giql_query)
print("Transpiled SQL:")
print(format_sql(sql))
print("\n" + "="*80 + "\n")

# Execute with DuckDB via GIQL engine
cursor = engine.conn.execute(sql)
result = cursor.df()  # Get result as pandas DataFrame
print(f"Result: {len(result)} intervals with cluster assignments")
print(f"Number of unique clusters: {result['cluster_id'].nunique()}")
result.head(10)

Transpiled SQL:
SELECT chromosome, start_pos, end_pos, signal_value, SUM(is_new_cluster) OVER (PARTITION BY "chromosome" ORDER BY "start_pos" NULLS LAST) AS cluster_id FROM (SELECT chromosome, start_pos, end_pos, signal_value, CASE WHEN LAG("end_pos") OVER (PARTITION BY "chromosome" ORDER BY "start_pos" NULLS LAST) >= "start_pos" THEN 0 ELSE 1 END AS is_new_cluster FROM features_a) AS lag_calc ORDER BY chromosome, start_pos


Result: 60893 intervals with cluster assignments
Number of unique clusters: 5495


,chromosome,start_pos,end_pos,signal_value,cluster_id
0,chr1,181368,181564,27.54796,1.0
1,chr1,186650,186846,19.93098,2.0
2,chr1,267909,268105,88.00863,3.0
3,chr1,586106,586302,35.02027,4.0
4,chr1,729261,729457,23.29415,5.0
5,chr1,778812,779008,56.97663,6.0
6,chr1,850473,850669,27.08243,7.0
7,chr1,858056,858252,15.55869,8.0
8,chr1,869860,869991,168.87294,9.0
9,chr1,904689,904883,167.56897,10.0


### CLUSTER with distance parameter

Cluster intervals within 1000bp of each other.

In [12]:
# Define GIQL query with distance parameter
giql_query = """
    SELECT
        chromosome,
        start_pos,
        end_pos,
        signal_value,
        CLUSTER(position, 10000) AS cluster_id
    FROM features_a
    ORDER BY chromosome, start_pos
"""

# Transpile to SQL
sql = engine.transpile(giql_query)
print("Transpiled SQL:")
print(format_sql(sql))
print("\n" + "="*80 + "\n")

# Execute with DuckDB via GIQL engine
cursor = engine.conn.execute(sql)
result = cursor.df()  # Get result as pandas DataFrame
print(f"Result: {len(result)} intervals with cluster assignments (1000bp distance)")
print(f"Number of unique clusters: {result['cluster_id'].nunique()}")
result.head(10)

Transpiled SQL:
SELECT chromosome, start_pos, end_pos, signal_value, SUM(is_new_cluster) OVER (PARTITION BY "chromosome" ORDER BY "start_pos" NULLS LAST) AS cluster_id FROM (SELECT chromosome, start_pos, end_pos, signal_value, CASE WHEN LAG("end_pos") OVER (PARTITION BY "chromosome" ORDER BY "start_pos" NULLS LAST) + 10000 >= "start_pos" THEN 0 ELSE 1 END AS is_new_cluster FROM features_a) AS lag_calc ORDER BY chromosome, start_pos


Result: 60893 intervals with cluster assignments (1000bp distance)
Number of unique clusters: 3692


,chromosome,start_pos,end_pos,signal_value,cluster_id
0,chr1,181368,181564,27.54796,1.0
1,chr1,186650,186846,19.93098,1.0
2,chr1,267909,268105,88.00863,2.0
3,chr1,586106,586302,35.02027,3.0
4,chr1,729261,729457,23.29415,4.0
5,chr1,778812,779008,56.97663,5.0
6,chr1,850473,850669,27.08243,6.0
7,chr1,858056,858252,15.55869,6.0
8,chr1,869860,869991,168.87294,7.0
9,chr1,904689,904883,167.56897,8.0


---

## 6. DISTANCE operator

**Calculate genomic distances** between intervals from `features_a` and `features_b`.

Similar to `bedtools closest -d`, this calculates the distance in base pairs between genomic intervals.

**GIQL Query**: `SELECT a.*, b.*, DISTANCE(a.position, b.position) AS distance FROM features_a a, features_b b`

In [13]:
# Define GIQL query with signed distance
# Signed distance (signed=true) returns negative values for upstream features and positive for downstream
# This is similar to bedtools closest -D ref
giql_query = """
    WITH distances AS (
        SELECT
            a.chromosome,
            a.start_pos AS a_start,
            a.end_pos AS a_end,
            b.start_pos AS b_start,
            b.end_pos AS b_end,
            DISTANCE(a.position, b.position, signed=true) AS signed_distance,
            ROW_NUMBER() OVER (
                PARTITION BY a.chromosome, a.start_pos, a.end_pos
                ORDER BY ABS(DISTANCE(a.position, b.position, signed=true))
            ) AS rank
        FROM features_a a
        CROSS JOIN features_b b
        WHERE a.chromosome = b.chromosome
            AND a.chromosome = 'chr1'
            AND a.start_pos < 500000
    )
    SELECT 
        chromosome,
        a_start,
        a_end,
        b_start,
        b_end,
        signed_distance,
        CASE 
            WHEN signed_distance < 0 THEN 'upstream'
            WHEN signed_distance > 0 THEN 'downstream'
            ELSE 'overlap'
        END AS direction
    FROM distances
    WHERE rank = 1
    ORDER BY chromosome, a_start
"""

# Transpile to SQL
sql = engine.transpile(giql_query)
print("Transpiled SQL:")
print(format_sql(sql))
print("\n" + "="*80 + "\n")

# Execute with DuckDB via GIQL engine
cursor = engine.conn.execute(sql)
result = cursor.df()  # Get result as pandas DataFrame
print(f"Result: {len(result)} features with directional distances")
print(f"Upstream features: {(result['signed_distance'] < 0).sum()}")
print(f"Downstream features: {(result['signed_distance'] > 0).sum()}")
print(f"Overlapping features: {(result['signed_distance'] == 0).sum()}")
result.head(10)

Transpiled SQL:
WITH distances AS (SELECT a.chromosome, a.start_pos AS a_start, a.end_pos AS a_end, b.start_pos AS b_start, b.end_pos AS b_end, CASE WHEN a."chromosome" != b."chromosome" THEN NULL WHEN a."start_pos" < b."end_pos" AND a."end_pos" > b."start_pos" THEN 0 WHEN a."end_pos" <= b."start_pos" THEN (b."start_pos" - a."end_pos") ELSE (a."start_pos" - b."end_pos") END AS signed_distance, ROW_NUMBER() OVER (PARTITION BY a.chromosome, a.start_pos, a.end_pos ORDER BY ABS(CASE WHEN a."chromosome" != b."chromosome" THEN NULL WHEN a."start_pos" < b."end_pos" AND a."end_pos" > b."start_pos" THEN 0 WHEN a."end_pos" <= b."start_pos" THEN (b."start_pos" - a."end_pos") ELSE (a."start_pos" - b."end_pos") END)) AS rank FROM features_a AS a CROSS JOIN features_b AS b WHERE a.chromosome = b.chromosome AND a.chromosome = 'chr1' AND a.start_pos < 500000) SELECT chromosome, a_start, a_end, b_start, b_end, signed_distance, CASE WHEN signed_distance < 0 THEN 'upstream' WHEN signed_distance > 0 THEN 

,chromosome,a_start,a_end,b_start,b_end,signed_distance,direction
0,chr1,181368,181564,186654,186964,5090,downstream
1,chr1,186650,186846,186654,186964,0,overlap
2,chr1,267909,268105,267979,268128,0,overlap


### Signed Distance (Directional)

Calculate **directional distances** where negative values indicate upstream features and positive values indicate downstream features, similar to `bedtools closest -D ref`.

In [14]:
# Define GIQL query - basic distance calculation
# DISTANCE() returns 0 for overlapping intervals, positive integers for gaps between intervals
# Filters to a small region on chr1 for demonstration
giql_query = """
    SELECT 
        a.chromosome,
        a.start_pos AS a_start,
        a.end_pos AS a_end,
        b.start_pos AS b_start,
        b.end_pos AS b_end,
        DISTANCE(a.position, b.position) AS distance
    FROM features_a a
    CROSS JOIN features_b b
    WHERE a.chromosome = b.chromosome 
        AND a.chromosome = 'chr1'
        AND a.start_pos BETWEEN 180000 AND 190000
        AND b.start_pos BETWEEN 180000 AND 200000
    ORDER BY a.start_pos, b.start_pos
"""

# Transpile to SQL
sql = engine.transpile(giql_query)
print("Transpiled SQL:")
print(format_sql(sql))
print("\n" + "="*80 + "\n")

# Execute with DuckDB via GIQL engine
cursor = engine.conn.execute(sql)
result = cursor.df()  # Get result as pandas DataFrame
print(f"Result: {len(result)} distance calculations")
print("\nExample distances (0 = overlap, positive = gap in base pairs):")
result.head(10)

Transpiled SQL:
SELECT a.chromosome, a.start_pos AS a_start, a.end_pos AS a_end, b.start_pos AS b_start, b.end_pos AS b_end, CASE WHEN a."chromosome" != b."chromosome" THEN NULL WHEN a."start_pos" < b."end_pos" AND a."end_pos" > b."start_pos" THEN 0 WHEN a."end_pos" <= b."start_pos" THEN (b."start_pos" - a."end_pos") ELSE (a."start_pos" - b."end_pos") END AS distance FROM features_a AS a CROSS JOIN features_b AS b WHERE a.chromosome = b.chromosome AND a.chromosome = 'chr1' AND a.start_pos BETWEEN 180000 AND 190000 AND b.start_pos BETWEEN 180000 AND 200000 ORDER BY a.start_pos, b.start_pos


Result: 2 distance calculations

Example distances (0 = overlap, positive = gap in base pairs):


,chromosome,a_start,a_end,b_start,b_end,distance
0,chr1,181368,181564,186654,186964,5090
1,chr1,186650,186846,186654,186964,0


### Finding Closest Features

Find the closest feature in B for each feature in A, replicating `bedtools closest -d` functionality.

In [15]:
# Define GIQL query to find closest feature in B for each feature in A
# This replicates bedtools closest -d by using ROW_NUMBER() to rank features by distance
giql_query = """
    WITH distances AS (
        SELECT
            a.chromosome,
            a.start_pos AS a_start,
            a.end_pos AS a_end,
            a.signal_value AS a_signal,
            b.start_pos AS b_start,
            b.end_pos AS b_end,
            b.signal_value AS b_signal,
            DISTANCE(a.position, b.position) AS distance,
            ROW_NUMBER() OVER (
                PARTITION BY a.chromosome, a.start_pos, a.end_pos
                ORDER BY DISTANCE(a.position, b.position)
            ) AS rank
        FROM features_a a
        CROSS JOIN features_b b
        WHERE a.chromosome = b.chromosome
            AND a.chromosome = 'chr1'
            AND a.start_pos < 1000000
    )
    SELECT 
        chromosome,
        a_start,
        a_end,
        a_signal,
        b_start,
        b_end,
        b_signal,
        distance
    FROM distances
    WHERE rank = 1
    ORDER BY chromosome, a_start
"""

# Transpile to SQL
sql = engine.transpile(giql_query)
print("Transpiled SQL:")
print(format_sql(sql))
print("\n" + "="*80 + "\n")

# Execute with DuckDB via GIQL engine
cursor = engine.conn.execute(sql)
result = cursor.df()  # Get result as pandas DataFrame
print(f"Result: {len(result)} features from A with their closest feature in B")
print(f"Average distance to closest feature: {result['distance'].mean():.1f} bp")
print(f"Features overlapping with closest: {(result['distance'] == 0).sum()}")
result.head(10)

Transpiled SQL:
WITH distances AS (SELECT a.chromosome, a.start_pos AS a_start, a.end_pos AS a_end, a.signal_value AS a_signal, b.start_pos AS b_start, b.end_pos AS b_end, b.signal_value AS b_signal, CASE WHEN a."chromosome" != b."chromosome" THEN NULL WHEN a."start_pos" < b."end_pos" AND a."end_pos" > b."start_pos" THEN 0 WHEN a."end_pos" <= b."start_pos" THEN (b."start_pos" - a."end_pos") ELSE (a."start_pos" - b."end_pos") END AS distance, ROW_NUMBER() OVER (PARTITION BY a.chromosome, a.start_pos, a.end_pos ORDER BY CASE WHEN a."chromosome" != b."chromosome" THEN NULL WHEN a."start_pos" < b."end_pos" AND a."end_pos" > b."start_pos" THEN 0 WHEN a."end_pos" <= b."start_pos" THEN (b."start_pos" - a."end_pos") ELSE (a."start_pos" - b."end_pos") END) AS rank FROM features_a AS a CROSS JOIN features_b AS b WHERE a.chromosome = b.chromosome AND a.chromosome = 'chr1' AND a.start_pos < 1000000) SELECT chromosome, a_start, a_end, a_signal, b_start, b_end, b_signal, distance FROM distances WHER

,chromosome,a_start,a_end,a_signal,b_start,b_end,b_signal,distance
0,chr1,181368,181564,27.54796,186654,186964,14.61908,5090
1,chr1,186650,186846,19.93098,186654,186964,14.61908,0
2,chr1,267909,268105,88.00863,267979,268128,49.39366,0
3,chr1,586106,586302,35.02027,850424,850734,15.07029,264122
4,chr1,729261,729457,23.29415,850424,850734,15.07029,120967
5,chr1,778812,779008,56.97663,850424,850734,15.07029,71416
6,chr1,850473,850669,27.08243,850424,850734,15.07029,0
7,chr1,858056,858252,15.55869,850424,850734,15.07029,7322
8,chr1,869860,869991,168.87294,869711,870021,12.42156,0
9,chr1,904689,904883,167.56897,912864,913174,31.07865,7981


---

## Summary

This demo showcased all 6 GIQL operators:

1. **INTERSECTS**: Binary predicate for overlapping intervals
2. **WITHIN**: Binary predicate for containment (A within B)
3. **CONTAINS**: Binary predicate for containment (A contains B)
4. **MERGE**: Aggregation operator to combine overlapping intervals
5. **CLUSTER**: Aggregation operator to assign cluster IDs to overlapping intervals
6. **DISTANCE**: UDF operator to calculate genomic distances between intervals

Each operator was:
- Written in GIQL syntax
- Transpiled to standard SQL
- Executed using DuckDB

This demonstrates how GIQL provides a high-level, intuitive syntax for genomic interval operations while maintaining compatibility with standard SQL engines through transpilation.

In [16]:
# Clean up
engine.close()